In [1]:
import os

In [2]:
%pwd

'c:\\Users\\layeg\\Desktop\\GitHub\\Holland_Barret\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\layeg\\Desktop\\GitHub\\Holland_Barret'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    """
    Configuration class for data ingestion.

    Attributes:
    root_dir (Path): The root directory where data will be stored.
    source_URL (str): The URL from which data will be downloaded.
    local_data_file (Path): The local file path where data will be stored after downloading.
    unzip_dir (Path): The directory where downloaded files will be unzipped.
    """
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    """
    Class for managing configuration files and retrieving data ingestion configuration.

    Attributes:
    config_filepath (str): The file path of the main configuration file.
    params_filepath (str): The file path of the parameters file.
    schema_filepath (str): The file path of the schema file.
    """

    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH
    ):
        """
        Initialize ConfigurationManager with file paths and read configuration files.

        Args:
        config_filepath (str): The file path of the main configuration file.
        params_filepath (str): The file path of the parameters file.
        schema_filepath (str): The file path of the schema file.
        """
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """
        Retrieve the data ingestion configuration from the main configuration.

        Returns:
        DataIngestionConfig: The data ingestion configuration object.
        """
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config


In [8]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [9]:
class DataIngestion:
    """
    Class for handling data ingestion tasks.

    Attributes:
    config (DataIngestionConfig): The configuration for data ingestion.
    """

    def __init__(self, config: DataIngestionConfig):
        """
        Initialize DataIngestion with a configuration object.

        Args:
        config (DataIngestionConfig): The configuration for data ingestion.
        """
        self.config = config

    def download_file(self):
        """
        Downloads a file from the specified URL if it doesn't already exist locally.
        """
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        Extracts the contents of a zip file into the specified directory.
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [10]:
try:
    # Initialize ConfigurationManager and retrieve data ingestion configuration
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()

    # Initialize DataIngestion with the retrieved configuration
    data_ingestion = DataIngestion(config=data_ingestion_config)

    # Download the file
    data_ingestion.download_file()

    # Extract the zip file
    data_ingestion.extract_zip_file()

except Exception as e:
    # Raise the caught exception
    raise e


[2024-02-28 11:14:40,172: INFO: common: YAML file loaded successfully from: config\config.yaml]
[2024-02-28 11:14:40,174: INFO: common: YAML file loaded successfully from: params.yaml]
[2024-02-28 11:14:40,177: INFO: common: YAML file loaded successfully from: schema.yaml]
[2024-02-28 11:14:40,179: INFO: common: Created directory at: artifacts]
[2024-02-28 11:14:40,180: INFO: common: Created directory at: artifacts/data_ingestion]
[2024-02-28 11:14:41,051: INFO: 2165076673: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 159202
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "21c6d50d4e866ed12fa4f0e2188bd8f7689ede13fe22ee7521ec9271ded41089"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: F414:903B4:3F7AD4E:42696F0:65DF15A0
Accept-Ranges: by

In [ ]:
# 